# Exports and plots

The sumo-experiments library implements some methods and functions to export and plot the results from a SUMO simulation. The original results files are encoded in an XML format, and this has two defaults : this format is not very human-readable, and the export capacities are low (especially for spreadsheet applications).

Once the data exported, it could be useful to plot them for sharing. The library implements a subpackage `plot` that easily plots data from results files.

## How to export data

First of all, let's do a simple experiment with default parameters.

In [ ]:
from sumo_experiments import Experiment
from sumo_experiments.preset_networks import OneCrossroadNetwork

net = OneCrossroadNetwork()
exp = Experiment(
    name = 'exports_and_plots',
    infrastructures = net.generate_infrastructures,
    flows = net.generate_flows_all_directions
)
exp.set_parameter('simulation_duration', 1000)
exp.run()

This script defines a SUMO network with only one intersection, managed by traffic lights, with flows coming from all directions, and leaving network by one of the three exits (a vehicle can't turn back). We set the `simulation_time` parameter to 1000, which implies that the simulation will end at 1000 steps.

We now have, in the current folder, a file `summary.xml`. This file contains data about all network like the mean waiting time of all vehicles, the mean speed or the mean travel time. This file is in XML format, but the `export_results_to_csv` method from the Experiment class will automatically transform it into a CSV file.

***Note :** The `summary.xml` file is deleted by default by the `clean_files` method. You can except this file by setting the parameter `except_summary` to `True`. However, we suggest you to first export all data and then delete all the files.* 

In [ ]:
exp.export_results_to_csv(
    filename = 'results.csv',
    sampling_rate = 50
)
exp.clean_files()

This method has two parameters :
- `filename` is the name of the created CSV file.
- `sampling_rate` is the the frequency of data export. For example, with a value of 50, the method will export the data at step 50, 100, 150, etc.

The CSV format is as following : each line represents an experiment, and not a sample. Samples are added in the columns. Here is a visual demonstration.

***Note :** We don't like that much this export format and it will probably be changed soon. It was originally adapted to be used with machine learning algorithms.*

In [ ]:
import pandas as pd

data = pd.read_csv('results.csv')
data

The behaviour of the `export_results_to_csv` function is to __append__ data to the file. It __does not overwrite__ the file. Let's generate another `results.csv` file, with the results of multiple experiments.

In [ ]:
!rm results.csv

green_times = [10, 20, 30, 40, 50, 60]

for green_time in green_times:
    
    exp = Experiment(
        name = f'exports_and_plots_{green_time}',
        infrastructures = net.generate_infrastructures,
        flows = net.generate_flows_all_directions
    )
    
    exp.set_parameter('green_time', green_time)
    exp.set_parameter('simulation_duration', 1000)
    
    exp.run()
    
    exp.export_results_to_csv(
        filename = 'results.csv',
        sampling_rate = 50
    )
    
    exp.clean_files()

In [ ]:
data = pd.read_csv('results.csv')
data

As you can see, the function appends data to the file. You can also see that the new parameter `green_time` has been added to the columns. The purpose of this export is to store data from comparable experiments, with __the same parameters__. If you try to export experiments with different parameters, the file will become unreadable for most of the spreadsheet softwares.

***Note :** It could be possible to store data with different parameters with a non-matrix format, like JSON or the original XML export format. We plan to create an export function in various other formats.*

## How to plot data

In this part, we'll see how to plot the exported data. The library implements a subpackage `plot` which contains a `ResultsPlotter` class. The class implements different methods to plot the mean wainting time, the mean travel time and the mean speed, for all vehicles.

***Note :** Some other functions are planned for the next updates.*

To plot the results, you first have to instanciate the class with some parameters :
- The path to the CSV file
- The sampling rate of export
- The number of simulation steps
- A folder where to save plots

In [ ]:
from sumo_experiments.plot import ResultsPlotter

rp = ResultsPlotter(
    file = 'results.csv',
    sampling_rate = 50,
    max_ticks = 1000,
    save_folder = '.'
)

Once the ResultsPlotter created, you can use its methods to create plots. Here is the current list :
- `plot_mean_waiting_time_n_worst` : Plot the evolution of mean waiting time for the experiments with the worst results
- `plot_mean_waiting_time_n_best` : Plot the evolution of mean waiting time for the experiments with the best results
- `plot_mean_travel_time_n_worst` : Plot the evolution of mean travel time for the experiments with the worst results
- `plot_mean_travel_time_n_best` : Plot the evolution of mean travel time for the experiments with the best results
- `plot_mean_speed_n_worst` : Plot the evolution of mean speed for the experiments with the worst results
- `plot_mean_speed_n_best` : Plot the evolution of mean speed for the experiments with the best results
- `plot_all` : Execute all of the previous methods

In [ ]:
rp.plot_mean_waiting_time_n_best(n=3)

In [ ]:
rp.plot_all(n_best=2, n_worst=2)

In [ ]:
!rm results.csv

In the next tutorial, we'll learn how to fully use the pre-set networks of the library to run experiments.